In [2]:
from torch import nn
import torch


In [38]:
logits = torch.Tensor([90.0,-10.0,90])
logits


tensor([ 90., -10.,  90.])

In [39]:
labels = torch.tensor([1.0, 0,1])

In [40]:
pred = logits.sigmoid()

In [41]:
bce_loss = nn.BCELoss()
bce_loss(pred,labels)

tensor(1.5133e-05)

In [32]:
pred

tensor([1.0000, 0.2689, 1.0000])

In [33]:
bce_loss = nn.BCEWithLogitsLoss()
bce_loss(logits,labels)

tensor(0.1044)